In [1]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

/aiffel/aiffel/aimmo/mmdetection


In [2]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [3]:
# # 완디비 로그인
wandb.login(key= '')

wandb: Currently logged in as: wodms. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [4]:
wandb.init(project="r-cnn", entity="aimmo")

wandb: Currently logged in as: wodms (aimmo). Use `wandb login --relogin` to force relogin


In [5]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_texts')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [6]:
config_file = 'configs/faster_rcnn/faster_rcnn_r101_caffe_fpn_1x_coco.py'
checkpoint_file = 'faster_rcnn_r101_caffe_fpn_1x_coco.pth'
cfg = Config.fromfile(config_file)

cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'


cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'middle_val.txt'
cfg.data.test.img_prefix = 'image'

cfg.model.roi_head.bbox_head.num_classes = 4

cfg.load_from = 'checkpoints/faster_rcnn_r101_caffe_fpn_1x_coco.pth'
cfg.work_dir = 'middle_fulldata'

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'

cfg.log_config.interval = 100
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
#cfg.evaluation.save_best='auto'
#cfg.data.train.pipeline[2].img_scale=[(960,520)]#['Resize']
#cfg.train_pipeline[2].img_scale=[(960,520)]
#cfg.test_pipeline[1].img_scale=[(960,520)]
#cfg.data.val.pipeline[1].img_scale = [(960,520)]
#cfg.data.test.pipeline[1].img_scale = [(960,520)]
cfg.runner.max_epochs = 15

cfg.data.samples_per_gpu = 4 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 0 # Worker to pre-fetch data for each single GPU
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet101_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntr

In [ ]:
 cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'r-cnn', 'entity' : 'aimmo', 'name' : 'r101'},)] 

In [8]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/aiffel/aiffel/aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(
2022-12-10 11:46:28,700 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-10 11:46:28,843 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r101_caffe_fpn_1x_coco.pth
2022-12-10 11:46:31,248 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_

2022-12-10 12:45:41,649 - mmdet - INFO - Epoch [1][2100/5226]	lr: 2.000e-02, eta: 1 day, 11:49:27, time: 1.688, data_time: 0.553, memory: 9593, loss_rpn_cls: 0.0232, loss_rpn_bbox: 0.0438, loss_cls: 0.1955, acc: 93.1187, loss_bbox: 0.2693, loss: 0.5318
2022-12-10 12:48:28,162 - mmdet - INFO - Epoch [1][2200/5226]	lr: 2.000e-02, eta: 1 day, 11:45:10, time: 1.665, data_time: 0.531, memory: 9593, loss_rpn_cls: 0.0243, loss_rpn_bbox: 0.0462, loss_cls: 0.1974, acc: 92.9790, loss_bbox: 0.2846, loss: 0.5526
2022-12-10 12:51:16,698 - mmdet - INFO - Epoch [1][2300/5226]	lr: 2.000e-02, eta: 1 day, 11:42:08, time: 1.685, data_time: 0.550, memory: 9593, loss_rpn_cls: 0.0220, loss_rpn_bbox: 0.0457, loss_cls: 0.2003, acc: 92.8218, loss_bbox: 0.2856, loss: 0.5535
2022-12-10 12:54:05,299 - mmdet - INFO - Epoch [1][2400/5226]	lr: 2.000e-02, eta: 1 day, 11:39:09, time: 1.686, data_time: 0.552, memory: 9593, loss_rpn_cls: 0.0261, loss_rpn_bbox: 0.0455, loss_cls: 0.1949, acc: 93.0601, loss_bbox: 0.2723, l

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 3.3 task/s, elapsed: 709s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-10 14:24:49,059 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 34396 | 0.964  | 0.925 |
| truck      | 3084  | 11311 | 0.890  | 0.776 |
| bus        | 1786  | 5322  | 0.924  | 0.845 |
| pedestrian | 3063  | 11668 | 0.850  | 0.709 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.814 |
+------------+-------+-------+--------+-------+
2022-12-10 14:24:49,136 - mmdet - INFO - Epoch(val) [1][2323]	AP50: 0.8140, mAP: 0.8136
2022-12-10 14:27:38,380 - mmdet - INFO - Epoch [2][100/5226]	lr: 2.000e-02, eta: 1 day, 9:56:34, time: 1.689, data_time: 0.553, memory: 9593, loss_rpn_cls: 0.0188, loss_rpn_bbox: 0.0435, loss_cls: 0.1765, acc: 93.5649, loss_bbox: 0.2691, loss: 0.5079
2022-12-10 14:30:26,269 - mmdet - INFO - Epoch [2][200/5226]	lr: 2.000e-02, eta: 1 day, 9:53:55, time: 1.679, data_time: 0.544, memory: 959

2022-12-10 15:53:55,578 - mmdet - INFO - Epoch [2][3200/5226]	lr: 2.000e-02, eta: 1 day, 8:29:08, time: 1.668, data_time: 0.532, memory: 9593, loss_rpn_cls: 0.0183, loss_rpn_bbox: 0.0396, loss_cls: 0.1687, acc: 93.7124, loss_bbox: 0.2521, loss: 0.4787
2022-12-10 15:56:42,663 - mmdet - INFO - Epoch [2][3300/5226]	lr: 2.000e-02, eta: 1 day, 8:26:21, time: 1.671, data_time: 0.537, memory: 9593, loss_rpn_cls: 0.0190, loss_rpn_bbox: 0.0394, loss_cls: 0.1671, acc: 93.8320, loss_bbox: 0.2470, loss: 0.4726
2022-12-10 15:59:29,318 - mmdet - INFO - Epoch [2][3400/5226]	lr: 2.000e-02, eta: 1 day, 8:23:29, time: 1.667, data_time: 0.532, memory: 9593, loss_rpn_cls: 0.0203, loss_rpn_bbox: 0.0418, loss_cls: 0.1649, acc: 94.0908, loss_bbox: 0.2526, loss: 0.4796
2022-12-10 16:02:16,886 - mmdet - INFO - Epoch [2][3500/5226]	lr: 2.000e-02, eta: 1 day, 8:20:46, time: 1.676, data_time: 0.541, memory: 9593, loss_rpn_cls: 0.0204, loss_rpn_bbox: 0.0413, loss_cls: 0.1628, acc: 94.0127, loss_bbox: 0.2461, loss:

...

2022-12-11 19:09:29,432 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 17603 | 0.969  | 0.958 |
| truck      | 3084  | 5165  | 0.933  | 0.900 |
| bus        | 1786  | 2716  | 0.958  | 0.937 |
| pedestrian | 3063  | 5773  | 0.887  | 0.837 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.908 |
+------------+-------+-------+--------+-------+
2022-12-11 19:09:29,504 - mmdet - INFO - Epoch(val) [12][2323]	AP50: 0.9080, mAP: 0.9080
2022-12-11 19:12:16,811 - mmdet - INFO - Epoch [13][100/5226]	lr: 2.000e-04, eta: 7:10:15, time: 1.670, data_time: 0.556, memory: 9593, loss_rpn_cls: 0.0030, loss_rpn_bbox: 0.0209, loss_cls: 0.0850, acc: 96.7261, loss_bbox: 0.1518, loss: 0.2607
2022-12-11 19:15:02,781 - mmdet - INFO - Epoch [13][200/5226]	lr: 2.000e-04, eta: 7:07:30, time: 1.660, data_time: 0.546, memory: 9593, loss_rpn

2022-12-11 20:38:19,553 - mmdet - INFO - Epoch [13][3200/5226]	lr: 2.000e-04, eta: 5:44:43, time: 1.665, data_time: 0.540, memory: 9593, loss_rpn_cls: 0.0029, loss_rpn_bbox: 0.0221, loss_cls: 0.0805, acc: 96.8257, loss_bbox: 0.1492, loss: 0.2547
2022-12-11 20:41:06,371 - mmdet - INFO - Epoch [13][3300/5226]	lr: 2.000e-04, eta: 5:41:57, time: 1.668, data_time: 0.532, memory: 9593, loss_rpn_cls: 0.0032, loss_rpn_bbox: 0.0209, loss_cls: 0.0812, acc: 96.8276, loss_bbox: 0.1499, loss: 0.2552
2022-12-11 20:43:53,675 - mmdet - INFO - Epoch [13][3400/5226]	lr: 2.000e-04, eta: 5:39:12, time: 1.673, data_time: 0.536, memory: 9593, loss_rpn_cls: 0.0026, loss_rpn_bbox: 0.0214, loss_cls: 0.0804, acc: 96.8550, loss_bbox: 0.1484, loss: 0.2528
2022-12-11 20:46:40,292 - mmdet - INFO - Epoch [13][3500/5226]	lr: 2.000e-04, eta: 5:36:26, time: 1.666, data_time: 0.534, memory: 9593, loss_rpn_cls: 0.0034, loss_rpn_bbox: 0.0211, loss_cls: 0.0802, acc: 96.8711, loss_bbox: 0.1472, loss: 0.2518
2022-12-11 20:49

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 3.3 task/s, elapsed: 706s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-11 21:46:33,468 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 17776 | 0.970  | 0.959 |
| truck      | 3084  | 5123  | 0.931  | 0.900 |
| bus        | 1786  | 2681  | 0.955  | 0.935 |
| pedestrian | 3063  | 5643  | 0.886  | 0.836 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.907 |
+------------+-------+-------+--------+-------+
2022-12-11 21:46:33,537 - mmdet - INFO - Epoch(val) [13][2323]	AP50: 0.9070, mAP: 0.9073
2022-12-11 21:49:22,662 - mmdet - INFO - Epoch [14][100/5226]	lr: 2.000e-04, eta: 4:45:56, time: 1.688, data_time: 0.552, memory: 9593, loss_rpn_cls: 0.0029, loss_rpn_bbox: 0.0196, loss_cls: 0.0744, acc: 97.0918, loss_bbox: 0.1429, loss: 0.2397
2022-12-11 21:52:10,140 - mmdet - INFO - Epoch [14][200/5226]	lr: 2.000e-04, eta: 4:43:11, time: 1.675, data_time: 0.538, memory: 9593, loss_rpn

KeyboardInterrupt: 